In [ ]:
!pip install --upgrade --quiet langchain-community langchain-openai neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os

# os.environ["OPENAI_API_KEY"] = ""
# os.environ["NEO4J_URI"] = ""
# os.environ["NEO4J_USERNAME"] = ""
# os.environ["NEO4J_PASSWORD"] = ""

In [ ]:
!pip install pandas

In [ ]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url=r"url_here",
    username="username_here",
    password=r"password_here",
)

In [ ]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Disease {identifier: STRING, kind: STRING, name: STRING}
Gene {identifier: INTEGER, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING, description: STRING, chromosome: STRING}
Anatomy {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Compound {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Symptom {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Pathway {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Molecular Function {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Cellular Component {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Biological Process {identifier: STRING, kind: STRING, name: STRING, source: STRING, license: STRING, url: STRING}
Principal Investigators {Prin

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4-turbo", temperature=0, openai_api_key='api_here')
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [ ]:
prompt_template = """
    make sure the keywords in the prompt are NOT case sensitive. make sure the labels with more than one word in the query are enclosed between ``.don't forget to return values (not keys) the result of the query. also remember to answer each question asked. make sure to pay careful attention to the directions of the relationships, while creating the cypher queries. return the full context of the returned result in human readable language.
    disease: investigated in -> study
    study: tests -> interventions
    interventions are key-value pairs in string format that have the following keys: DRUG, PROCEDURE, DEVICE, BEHAVIOUR, DIAGNOSTIC_TEST, BIOLOGICAL, RADIATION, COMBINATION, PRODUCT, GENETIC, DIETARY_SUPPLEMENT, OTHER. If the query is asking about drugs, then look for the drug here.
    disease: associates/upregulates/downregulates -> gene
    studies: led by -> principal investigators
    disease: studied by -> principal investigators
    compounds: treat/palliate -> disease
    phases in the knowledge graph are in capital letters with no gap between the word phase and the phase number, such as PHASE1, PHASE2, PHASE3, PHASE4
    principal investigators in the graph are under the label 'Principal Investigator'. For questions about principal investigators, the final return statement should be 'RETURN pi'. disease is studied by principal investigators.
    gene: participate -> `Biological Process`, `Cellular Component`, `Molecular Function`, and `Pathway`.
    consider all relations bidirectional
    principal investigators and drugs are directly connected through study. there's no need to go through disease.
"""

query = 'Which principal investigator led studies in which Dexmedetomidine was an intervention?'


In [ ]:
%%capture cellop
response = chain.invoke({"query": (prompt_template+query)})

# response['result']

In [ ]:
print("cellop: ", cellop())

parts = cellop.stdout.split('Full Context:')[1]
parts

print("parts: ", parts)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Study)-[:tests]->(i:Interventions)
WHERE i.`Interventions 1` CONTAINS "Dexmedetomidine" OR 
      i.`Interventions 2` CONTAINS "Dexmedetomidine" OR 
      i.`Interventions 3` CONTAINS "Dexmedetomidine" OR 
      i.`Interventions 4` CONTAINS "Dexmedetomidine" OR 
      i.`Interventions 5` CONTAINS "Dexmedetomidine"
MATCH (s)-[:led_by]->(pi:`Principal Investigators`)
RETURN pi

Full Context:
[{'pi': {'Principal Investigators 2': 'Pekka Talke'}}]

> Finished chain.
cellop:  None
parts:  
[{'pi': {'Principal Investigators 2': 'Pekka Talke'}}]

> Finished chain.



In [ ]:
# response['result']

In [ ]:
from openai import OpenAI

OPENAI_API_KEY = 'api_here'
client = OpenAI(api_key=OPENAI_API_KEY)
prompt = "I am going to give you a question I asked, and the answer to the question which I already have. If the answer is not blank, then return the answer in human readable natural language form. If the answer is blank, then explain that the query made to the knowledge graph did not return any relevant answers. Here is the question: "+query+ "Here is the answer: "+parts

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a clinical research assistant"},
    {"role": "user", "content": prompt}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='The principal investigator who led studies involving Dexmedetomidine as an intervention was Pekka Talke.', role='assistant', function_call=None, tool_calls=None)
